In [1]:
import pandas as pd
import vtk

import vtkmodules.vtkInteractionStyle
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkFiltersSources import vtkConeSource
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderer
)

#Define Colors

colors = vtkNamedColors()

# Read data
trees = pd.read_csv('../../Structures/trees_baseline.txt', sep='\t')

In [2]:
#Function Create Tree

def create_tree(x, y, z, id):
    # Define the eight vertices of the cube
    points = vtk.vtkPoints()
    points.InsertNextPoint(x[0], y[0], z)  # 0: bottom-left-front
    points.InsertNextPoint(x[1], y[1], z)  # 1: bottom-right-front
    points.InsertNextPoint(x[2], y[2], z)  # 2: top-right-front
    points.InsertNextPoint(x[3], y[3], z)  # 3: top-left-front
    points.InsertNextPoint(x[0], y[0], 0)  # 4: bottom-left-back
    points.InsertNextPoint(x[1], y[1], 0)  # 5: bottom-right-back
    points.InsertNextPoint(x[2], y[2], 0)  # 6: top-right-back
    points.InsertNextPoint(x[3], y[3], 0)  # 7: top-left-back

    # Define the six faces of the cube (topology)
    faces = vtk.vtkCellArray()

    face = vtk.vtkPolygon()
    face.GetPointIds().SetNumberOfIds(4)
    for j in range(4):  # bottom face
        face.GetPointIds().SetId(j, j + 4)
    faces.InsertNextCell(face)

    face = vtk.vtkPolygon()
    face.GetPointIds().SetNumberOfIds(4)
    for j in range(4):  # top face
        face.GetPointIds().SetId(j, j)
    faces.InsertNextCell(face)

    for i in range(4):  # side faces
        face = vtk.vtkPolygon()
        face.GetPointIds().SetNumberOfIds(4)
        face.GetPointIds().SetId(0, i)
        face.GetPointIds().SetId(1, (i + 1) % 4)
        face.GetPointIds().SetId(2, (i + 1) % 4 + 4)
        face.GetPointIds().SetId(3, i + 4)
        faces.InsertNextCell(face)

    # Create a polydata object
    cube = vtk.vtkPolyData()

    # Set the points and polygons
    cube.SetPoints(points)
    cube.SetPolys(faces)

    # Create a mapper and actor
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(cube)

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(colors.GetColor3d('Green'))


    return actor


In [3]:
# Initialize the renderer
renderer = vtk.vtkRenderer()
renderer.SetBackground(colors.GetColor3d('LightBlue'))


# For each tree id, create the corresponding actor and add it to the renderer
for id in trees['id'].unique():
    tree_data = trees[trees['id'] == id]
    x = tree_data['x'].values
    y = tree_data['y'].values
    z = tree_data['z'].values[0]  # height is the same for all points of a building
    actor = create_tree(x, y, z, id)
    renderer.AddActor(actor)


In [4]:

# Create a render window and add the renderer to it
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

In [5]:
# Create a render window interactor and set the render window for it
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Start the visualization
interactor.Initialize()
interactor.Start()